# Step D - NB01: Foundation

**Constraint-Based Architectural NCA - Step D**

**Version:** 1.0
**Date:** December 2025
**Purpose:** Build core components for Step D - connecting 3 access points between buildings

---

## Scene Definition

- 2 buildings, each >=20m tall
- Gap >= 18m between buildings
- Gap zones: 3m pedestrian | 12m no-go | 3m pedestrian
- 3 access points: 2 on building facades, 1 on ground (in pedestrian zone)

## Key Constraints

- No structure inside buildings
- Ground zone (0-5m) empty except around ground access
- No-go zone (12m center) forbidden below 5m
- Height ceiling: tallest building + 2m
- Volume: 20-40% of one building
- All 3 access points must be connected

---

## 1. Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_ROOT = '/content/drive/MyDrive/Constraint-NCA-StepD'
os.makedirs(f'{PROJECT_ROOT}/checkpoints', exist_ok=True)
os.makedirs(f'{PROJECT_ROOT}/logs', exist_ok=True)
print(f'Project root: {PROJECT_ROOT}')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from typing import Dict, Tuple, List
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

## 2. Configuration

In [ ]:
# Step D Configuration
# Using 1 voxel = 1 meter for simplicity

CONFIG = {
    # Grid - larger to accommodate 20m+ buildings and 18m+ gap
    'grid_size': 64,           # 64m x 64m x 64m volume
    'n_channels': 8,
    'n_frozen': 4,
    'n_grown': 4,

    # Channel indices
    'ch_ground': 0,
    'ch_existing': 1,
    'ch_access': 2,
    'ch_zones': 3,             # Zone masks (pedestrian, no-go, etc.)
    'ch_structure': 4,
    'ch_alive': 5,
    'ch_hidden1': 6,
    'ch_hidden2': 7,

    # Scene parameters
    'min_building_height': 20,  # meters
    'max_building_height': 30,
    'min_gap': 18,              # meters
    'max_gap': 24,
    'pedestrian_zone': 3,       # meters on each side
    'ground_clearance': 5,      # meters - keep empty below this
    'ceiling_margin': 2,        # meters above tallest building

    # Volume constraints
    'min_volume_ratio': 0.20,   # 20% of building volume
    'max_volume_ratio': 0.40,   # 40% of building volume

    # Network
    'hidden_dim': 128,
    'fire_rate': 0.5,
    'update_scale': 0.1,

    # Initialization
    'xavier_gain': 0.5,
    'structure_bias': 0.05,

    # Training
    'batch_size': 2,            # Larger grid = smaller batch
    'lr_initial': 1e-3,
    'grad_clip': 1.0,
}

print('Step D Configuration')
print('='*50)
print(f"Grid: {CONFIG['grid_size']}^3 (1 voxel = 1 meter)")
print(f"Buildings: {CONFIG['min_building_height']}-{CONFIG['max_building_height']}m tall")
print(f"Gap: {CONFIG['min_gap']}-{CONFIG['max_gap']}m")
print(f"Pedestrian zone: {CONFIG['pedestrian_zone']}m each side")
print(f"Ground clearance: 0-{CONFIG['ground_clearance']}m must be empty")
print(f"Volume: {CONFIG['min_volume_ratio']*100:.0f}-{CONFIG['max_volume_ratio']*100:.0f}% of building")
print('='*50)

In [ ]:
# Save config
with open(f'{PROJECT_ROOT}/config_step_d.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)
print('Config saved')

## 3. Perception Module

In [ ]:
class Perceive3D(nn.Module):
    """3D Sobel perception with replicate padding."""

    def __init__(self, n_channels: int = 8):
        super().__init__()
        self.n_channels = n_channels

        sobel_x = self._create_sobel_kernel('x')
        sobel_y = self._create_sobel_kernel('y')
        sobel_z = self._create_sobel_kernel('z')
        identity = self._create_identity_kernel()

        kernels = torch.stack([identity, sobel_x, sobel_y, sobel_z], dim=0)
        self.register_buffer('kernels', kernels)

    def _create_sobel_kernel(self, direction: str) -> torch.Tensor:
        derivative = torch.tensor([-1., 0., 1.])
        smoothing = torch.tensor([1., 2., 1.])

        if direction == 'x':
            kernel = torch.einsum('i,j,k->ijk', smoothing, smoothing, derivative)
        elif direction == 'y':
            kernel = torch.einsum('i,j,k->ijk', smoothing, derivative, smoothing)
        elif direction == 'z':
            kernel = torch.einsum('i,j,k->ijk', derivative, smoothing, smoothing)
        return kernel / 16.0

    def _create_identity_kernel(self) -> torch.Tensor:
        kernel = torch.zeros(3, 3, 3)
        kernel[1, 1, 1] = 1.0
        return kernel

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, D, H, W = x.shape
        x_padded = F.pad(x, (1, 1, 1, 1, 1, 1), mode='replicate')
        outputs = []
        for k in range(4):
            kernel = self.kernels[k:k+1].unsqueeze(0).expand(C, 1, 3, 3, 3)
            out = F.conv3d(x_padded, kernel, padding=0, groups=C)
            outputs.append(out)
        return torch.cat(outputs, dim=1)


# Test
print('Testing Perceive3D...')
perceive = Perceive3D(CONFIG['n_channels']).to(device)
test_input = torch.randn(1, CONFIG['n_channels'], 16, 16, 16, device=device)
test_output = perceive(test_input)
print(f'  Input: {test_input.shape}')
print(f'  Output: {test_output.shape}')
print('  OK')

## 4. NCA Model

In [ ]:
class StepDNCA(nn.Module):
    """Neural Cellular Automaton for Step D.
    
    Grows structure to connect 3 access points while respecting:
    - Building exclusion
    - Ground zone clearance
    - No-go zones
    - Height ceiling
    - Volume budget
    """

    def __init__(self, config: dict):
        super().__init__()
        self.config = config

        n_channels = config['n_channels']
        hidden_dim = config['hidden_dim']
        perception_dim = n_channels * 4
        n_grown = config['n_grown']

        self.perceive = Perceive3D(n_channels)

        self.update_net = nn.Sequential(
            nn.Conv3d(perception_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, hidden_dim, 1),
            nn.ReLU(),
            nn.Conv3d(hidden_dim, n_grown, 1),
        )

        self._init_weights()

    def _init_weights(self):
        gain = self.config['xavier_gain']
        for m in self.update_net:
            if isinstance(m, nn.Conv3d):
                nn.init.xavier_uniform_(m.weight, gain=gain)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        # Small positive bias for structure channel
        last_layer = self.update_net[-1]
        with torch.no_grad():
            last_layer.bias[0] = self.config['structure_bias']

    def forward(self, state: torch.Tensor, steps: int = 1) -> torch.Tensor:
        for _ in range(steps):
            state = self._step(state)
        return state

    def _step(self, state: torch.Tensor) -> torch.Tensor:
        B, C, D, H, W = state.shape
        cfg = self.config

        perception = self.perceive(state)
        delta = self.update_net(perception)

        # Stochastic update
        if self.training:
            fire_mask = (torch.rand(B, 1, D, H, W, device=state.device) < cfg['fire_rate']).float()
            delta = delta * fire_mask

        # Update grown channels only
        grown_start = cfg['n_frozen']
        grown_new = state[:, grown_start:] + cfg['update_scale'] * delta
        grown_new = torch.clamp(grown_new, 0.0, 1.0)

        # Hard mask: no structure inside existing buildings
        existing = state[:, cfg['ch_existing']:cfg['ch_existing']+1]
        available = 1.0 - existing
        struct_new = grown_new[:, 0:1] * available
        grown_masked = torch.cat([struct_new, grown_new[:, 1:]], dim=1)

        return torch.cat([state[:, :grown_start], grown_masked], dim=1)

    def grow(self, seed: torch.Tensor, steps: int = 50) -> torch.Tensor:
        """Grow structure from seed state."""
        self.eval()
        with torch.no_grad():
            return self.forward(seed, steps)


# Test
print('Testing StepDNCA...')
model = StepDNCA(CONFIG).to(device)
params = sum(p.numel() for p in model.parameters())
print(f'  Parameters: {params:,}')
test_state = torch.zeros(1, CONFIG['n_channels'], 32, 32, 32, device=device)
test_out = model(test_state, steps=1)
print(f'  Input: {test_state.shape}')
print(f'  Output: {test_out.shape}')
print('  OK')

## 5. Scene Generator

In [ ]:
class StepDSceneGenerator:
    """Generate Step D scenes with 2 buildings and 3 access points.
    
    Scene layout:
        [Building A] | 3m ped | 12m no-go | 3m ped | [Building B]
    
    Access points:
        - AP1: Building A facade (facing gap), height H1
        - AP2: Building B facade (facing gap), height H2 != H1
        - AP3: Ground, in pedestrian zone
    """

    def __init__(self, config: dict):
        self.config = config
        self.G = config['grid_size']

    def generate(self, difficulty: str = 'easy', device: str = 'cuda') -> Tuple[torch.Tensor, dict]:
        """Generate a scene.
        
        Returns:
            state: (1, C, D, H, W) tensor
            metadata: dict with scene info
        """
        G = self.G
        cfg = self.config
        state = torch.zeros(1, cfg['n_channels'], G, G, G, device=device)

        # Ground plane
        state[:, cfg['ch_ground'], 0, :, :] = 1.0

        # Get difficulty parameters
        params = self._get_params(difficulty)

        # Place buildings
        building_info = self._place_buildings(state, params)

        # Place access points
        access_info = self._place_access_points(state, params, building_info)

        # Create zone masks
        zone_info = self._create_zone_masks(state, params, building_info)

        metadata = {
            'difficulty': difficulty,
            'buildings': building_info,
            'access_points': access_info,
            'zones': zone_info,
            'params': params,
        }

        return state, metadata

    def _get_params(self, difficulty: str) -> dict:
        """Get scene parameters based on difficulty."""
        cfg = self.config
        
        if difficulty == 'easy':
            return {
                'height_A': 20,
                'height_B': 20,
                'gap': 18,
                'building_width': 12,
                'building_depth': 20,
                'ap1_height': 10,  # Same height for easy
                'ap2_height': 10,
            }
        elif difficulty == 'medium':
            return {
                'height_A': random.randint(20, 25),
                'height_B': random.randint(20, 25),
                'gap': random.randint(18, 22),
                'building_width': random.randint(10, 14),
                'building_depth': random.randint(18, 24),
                'ap1_height': random.randint(8, 15),
                'ap2_height': random.randint(8, 15),
            }
        else:  # hard
            h1 = random.randint(20, 30)
            h2 = random.randint(20, 30)
            return {
                'height_A': h1,
                'height_B': h2,
                'gap': random.randint(18, 24),
                'building_width': random.randint(8, 14),
                'building_depth': random.randint(16, 26),
                'ap1_height': random.randint(6, h1 - 4),
                'ap2_height': random.randint(6, h2 - 4),
            }

    def _place_buildings(self, state: torch.Tensor, params: dict) -> dict:
        """Place two buildings with gap between them."""
        G = self.G
        cfg = self.config
        ch = cfg['ch_existing']

        gap = params['gap']
        w = params['building_width']
        d = params['building_depth']
        h_A = params['height_A']
        h_B = params['height_B']

        # Center the scene
        gap_center = G // 2
        y_start = (G - d) // 2

        # Building A (left)
        x_A_end = gap_center - gap // 2
        x_A_start = x_A_end - w
        state[:, ch, :h_A, y_start:y_start+d, x_A_start:x_A_end] = 1.0

        # Building B (right)
        x_B_start = gap_center + gap // 2
        x_B_end = x_B_start + w
        state[:, ch, :h_B, y_start:y_start+d, x_B_start:x_B_end] = 1.0

        return {
            'A': {
                'x': (x_A_start, x_A_end),
                'y': (y_start, y_start + d),
                'z': (0, h_A),
                'height': h_A,
                'volume': h_A * w * d,
                'facade_x': x_A_end,  # Facade facing gap
            },
            'B': {
                'x': (x_B_start, x_B_end),
                'y': (y_start, y_start + d),
                'z': (0, h_B),
                'height': h_B,
                'volume': h_B * w * d,
                'facade_x': x_B_start,  # Facade facing gap
            },
            'gap': {
                'x': (x_A_end, x_B_start),
                'width': gap,
                'center': gap_center,
            },
            'max_height': max(h_A, h_B),
        }

    def _place_access_points(self, state: torch.Tensor, params: dict, buildings: dict) -> list:
        """Place 3 access points."""
        G = self.G
        cfg = self.config
        ch = cfg['ch_access']
        ped_zone = cfg['pedestrian_zone']

        gap_info = buildings['gap']
        y_mid = (buildings['A']['y'][0] + buildings['A']['y'][1]) // 2

        access_points = []

        # AP1: Building A facade
        ap1_x = buildings['A']['facade_x']  # Just at facade edge
        ap1_y = y_mid
        ap1_z = params['ap1_height']
        state[:, ch, ap1_z:ap1_z+2, ap1_y-1:ap1_y+2, ap1_x:ap1_x+2] = 1.0
        access_points.append({
            'id': 'AP1',
            'type': 'building_A',
            'x': ap1_x, 'y': ap1_y, 'z': ap1_z,
        })

        # AP2: Building B facade
        ap2_x = buildings['B']['facade_x'] - 1  # Just at facade edge
        ap2_y = y_mid
        ap2_z = params['ap2_height']
        state[:, ch, ap2_z:ap2_z+2, ap2_y-1:ap2_y+2, ap2_x-1:ap2_x+1] = 1.0
        access_points.append({
            'id': 'AP2',
            'type': 'building_B',
            'x': ap2_x, 'y': ap2_y, 'z': ap2_z,
        })

        # AP3: Ground in pedestrian zone
        # Choose randomly: near building A or B
        if random.random() < 0.5:
            # Near building A (in its pedestrian zone)
            ap3_x = buildings['A']['facade_x'] + ped_zone // 2
        else:
            # Near building B (in its pedestrian zone)
            ap3_x = buildings['B']['facade_x'] - ped_zone // 2 - 1
        
        ap3_y = y_mid + random.randint(-3, 3)
        ap3_z = 0
        state[:, ch, ap3_z:ap3_z+2, ap3_y-1:ap3_y+2, ap3_x-1:ap3_x+2] = 1.0
        access_points.append({
            'id': 'AP3',
            'type': 'ground',
            'x': ap3_x, 'y': ap3_y, 'z': ap3_z,
        })

        return access_points

    def _create_zone_masks(self, state: torch.Tensor, params: dict, buildings: dict) -> dict:
        """Create zone masks stored in ch_zones.
        
        Zone encoding in ch_zones:
        - 0.0: Available space
        - 0.25: Pedestrian zone (0-5m only)
        - 0.5: No-go zone (0-5m only)
        - 0.75: Above ceiling
        - 1.0: Inside building (blocked)
        """
        G = self.G
        cfg = self.config
        ch = cfg['ch_zones']
        ped_zone = cfg['pedestrian_zone']
        clearance = cfg['ground_clearance']
        ceiling_margin = cfg['ceiling_margin']

        gap_info = buildings['gap']
        max_height = buildings['max_height']
        ceiling_height = max_height + ceiling_margin

        # Start with zeros (available)
        zones = state[:, ch]

        # Buildings are blocked (1.0) - already in ch_existing
        existing = state[:, cfg['ch_existing']]
        zones = zones + existing  # Will be clamped later

        # Above ceiling (0.75)
        zones[:, ceiling_height:, :, :] = 0.75

        # Ground zone (0-5m) in gap area
        gap_x_start = gap_info['x'][0]
        gap_x_end = gap_info['x'][1]
        
        # Pedestrian zones (0.25)
        ped_A_end = gap_x_start + ped_zone
        ped_B_start = gap_x_end - ped_zone
        zones[:, :clearance, :, gap_x_start:ped_A_end] = 0.25
        zones[:, :clearance, :, ped_B_start:gap_x_end] = 0.25

        # No-go zone (0.5) - center of gap below 5m
        zones[:, :clearance, :, ped_A_end:ped_B_start] = 0.5

        state[:, ch] = torch.clamp(zones, 0, 1)

        return {
            'pedestrian_A': (gap_x_start, ped_A_end),
            'pedestrian_B': (ped_B_start, gap_x_end),
            'nogo': (ped_A_end, ped_B_start),
            'ground_clearance': clearance,
            'ceiling': ceiling_height,
        }

    def batch(self, difficulty: str, batch_size: int, device: str) -> torch.Tensor:
        """Generate batch of scenes."""
        scenes = [self.generate(difficulty, device)[0] for _ in range(batch_size)]
        return torch.cat(scenes, dim=0)


# Test
print('Testing StepDSceneGenerator...')
gen = StepDSceneGenerator(CONFIG)
scene, meta = gen.generate('easy', device)
print(f'  Scene shape: {scene.shape}')
print(f'  Building A: height={meta["buildings"]["A"]["height"]}m, volume={meta["buildings"]["A"]["volume"]}')
print(f'  Building B: height={meta["buildings"]["B"]["height"]}m')
print(f'  Gap: {meta["buildings"]["gap"]["width"]}m')
print(f'  Access points:')
for ap in meta['access_points']:
    print(f'    {ap["id"]}: {ap["type"]} at z={ap["z"]}')
print(f'  Zones: ceiling={meta["zones"]["ceiling"]}m')
print('  OK')

## 6. Visualization

In [ ]:
def visualize_scene(state: torch.Tensor, metadata: dict, config: dict,
                    title: str = 'Step D Scene', show_zones: bool = True):
    """Visualize Step D scene - ZOOMED to region of interest."""
    cfg = config
    s = state[0].cpu().numpy()
    G = s.shape[1]

    existing = s[cfg['ch_existing']] > 0.5
    access = s[cfg['ch_access']] > 0.5
    structure = s[cfg['ch_structure']] > 0.5
    zones = s[cfg['ch_zones']]

    # Calculate bounding box from buildings + margin
    buildings = metadata['buildings']
    margin = 4  # meters margin around scene

    x_min = max(0, buildings['A']['x'][0] - margin)
    x_max = min(G, buildings['B']['x'][1] + margin)
    y_min = max(0, buildings['A']['y'][0] - margin)
    y_max = min(G, buildings['A']['y'][1] + margin)
    z_min = 0
    z_max = min(G, metadata['zones']['ceiling'] + margin)

    # Crop arrays to region of interest
    existing_crop = existing[z_min:z_max, y_min:y_max, x_min:x_max]
    access_crop = access[z_min:z_max, y_min:y_max, x_min:x_max]
    structure_crop = structure[z_min:z_max, y_min:y_max, x_min:x_max]

    fig = plt.figure(figsize=(20, 6))

    # 3D view - CROPPED
    ax1 = fig.add_subplot(141, projection='3d')
    if existing_crop.any():
        ax1.voxels(existing_crop.transpose(1, 2, 0), facecolors='gray', alpha=0.3)
    if access_crop.any():
        ax1.voxels(access_crop.transpose(1, 2, 0), facecolors='green', alpha=0.9)
    if structure_crop.any():
        ax1.voxels(structure_crop.transpose(1, 2, 0), facecolors='royalblue', alpha=0.6)
    ax1.set_xlabel('Y (m)')
    ax1.set_ylabel('X (m)')
    ax1.set_zlabel('Z (m)')
    ax1.set_title(title)
    ax1.view_init(elev=25, azim=45)

    # Plan view (top-down) - CROPPED
    # Data: plan[y, x] - Y is rows (vertical), X is columns (horizontal)
    ax2 = fig.add_subplot(142)
    d_crop = y_max - y_min
    w_crop = x_max - x_min
    plan = np.zeros((d_crop, w_crop, 3))
    exist_plan = existing_crop.max(axis=0)  # collapse Z -> (Y, X)
    struct_plan = structure_crop.max(axis=0)
    access_plan = access_crop.max(axis=0)
    plan[exist_plan] = [0.5, 0.5, 0.5]
    if struct_plan.any():
        plan[struct_plan] = [0.2, 0.4, 0.8]
    plan[access_plan] = [0.2, 0.8, 0.2]
    # NO transpose - Y vertical, X horizontal
    ax2.imshow(plan, origin='lower', extent=[x_min, x_max, y_min, y_max])
    ax2.set_title('Plan View')
    ax2.set_xlabel('X (m)')
    ax2.set_ylabel('Y (m)')

    # Elevation view - CROPPED
    # Data: elev[z, x] - Z is rows (vertical), X is columns (horizontal)
    ax3 = fig.add_subplot(143)
    z_crop = z_max - z_min
    elev = np.zeros((z_crop, w_crop, 3))
    exist_elev = existing_crop.max(axis=1)  # collapse Y -> (Z, X)
    struct_elev = structure_crop.max(axis=1)
    access_elev = access_crop.max(axis=1)
    elev[exist_elev] = [0.5, 0.5, 0.5]
    if struct_elev.any():
        elev[struct_elev] = [0.2, 0.4, 0.8]
    elev[access_elev] = [0.2, 0.8, 0.2]
    # Show no-go zone (adjusted for crop)
    if show_zones:
        clearance = metadata['zones']['ground_clearance']
        nogo = metadata['zones']['nogo']
        nogo_x_start = max(0, nogo[0] - x_min)
        nogo_x_end = min(w_crop, nogo[1] - x_min)
        for x in range(nogo_x_start, nogo_x_end):
            for z in range(min(clearance, z_crop)):
                if elev[z, x, 0] == 0:
                    elev[z, x] = [1.0, 0.8, 0.8]
    # NO transpose - Z vertical, X horizontal
    ax3.imshow(elev, origin='lower', extent=[x_min, x_max, z_min, z_max])
    # Horizontal lines at constant Z values
    ax3.axhline(y=metadata['zones']['ceiling'], color='r', linestyle='--', alpha=0.7, label='Ceiling')
    ax3.axhline(y=metadata['zones']['ground_clearance'], color='orange', linestyle='--', alpha=0.7, label='Ground clearance')
    ax3.legend(loc='upper right', fontsize=8)
    ax3.set_title('Elevation (red=no-go zone)')
    ax3.set_xlabel('X (m)')
    ax3.set_ylabel('Z (m)')

    # Info panel
    ax4 = fig.add_subplot(144)
    ax4.axis('off')

    zones_info = metadata['zones']
    struct_vol = structure.sum()
    ref_vol = buildings['A']['volume']
    vol_ratio = struct_vol / ref_vol if ref_vol > 0 else 0

    info_text = f"""SCENE INFO (1 voxel = 1m)

Building A: {buildings['A']['height']}m tall
Building B: {buildings['B']['height']}m tall
Gap: {buildings['gap']['width']}m

Access Points:
"""
    for ap in metadata['access_points']:
        info_text += f"  {ap['id']}: z={ap['z']}m ({ap['type']})\n"

    info_text += f"""
Zones:
  Ground clearance: 0-{zones_info['ground_clearance']}m
  Ceiling: {zones_info['ceiling']}m
  No-go (center): {zones_info['nogo'][1] - zones_info['nogo'][0]}m

Structure:
  Volume: {struct_vol:.0f} voxels ({struct_vol:.0f} m3)
  Ratio: {vol_ratio*100:.1f}% of Building A
  Target: 20-40%
"""
    ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes,
             fontsize=10, va='top', family='monospace')

    plt.tight_layout()
    plt.show()


# Test visualization
print('Visualizing test scene (ZOOMED)...')
visualize_scene(scene, meta, CONFIG, 'Test Scene (Easy)')

In [ ]:
# Test different difficulties
print('Medium difficulty:')
scene_med, meta_med = gen.generate('medium', device)
visualize_scene(scene_med, meta_med, CONFIG, 'Test Scene (Medium)')

print('Hard difficulty:')
scene_hard, meta_hard = gen.generate('hard', device)
visualize_scene(scene_hard, meta_hard, CONFIG, 'Test Scene (Hard)')

## 7. Test NCA Growth

In [ ]:
# Test untrained NCA growth
print('Testing untrained NCA growth...')
model = StepDNCA(CONFIG).to(device)

scene, meta = gen.generate('easy', device)

with torch.no_grad():
    grown = model.grow(scene, steps=50)

print(f'Structure mean: {grown[0, CONFIG["ch_structure"]].mean().item():.4f}')
visualize_scene(grown, meta, CONFIG, 'Untrained NCA (50 steps)')

## 8. Summary

### Components Created

1. **CONFIG** - Step D configuration with 64^3 grid, 20m+ buildings, 18m+ gap
2. **Perceive3D** - 3D Sobel perception module
3. **StepDNCA** - NCA model with building exclusion mask
4. **StepDSceneGenerator** - Generates scenes with:
   - 2 buildings of configurable height
   - 3 access points (2 on facades, 1 on ground)
   - Zone masks (pedestrian, no-go, ceiling)
5. **visualize_scene** - Visualization with zone highlighting

### Next: NB02_Training

Implement loss functions and training loop.

In [ ]:
print('NB01 Foundation complete!')
print(f'Config saved to: {PROJECT_ROOT}/config_step_d.json')